# fairlearn Trial

fairnessを実現するためのmethodを実行。

- CorrelationRemover
- GridSearch
- ThresholdOptimizer

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.base import clone
from lightgbm import LGBMClassifier
%matplotlib inline

In [2]:
# fairlearnで用いるmodel関連methodのimport
from fairlearn.preprocessing import CorrelationRemover
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.reductions import DemographicParity, EqualizedOdds, TruePositiveRateParity
from fairlearn.reductions import GridSearch

In [3]:
def logistic(x, a=1):
    return 1/(1 + np.exp(-a*x))

# サンプルデータ生成
def make_sample_data(N=10000, p_s=0.1):
    s = np.r_[np.zeros(int(N*p_s)) , np.ones(int(N*(1-p_s)))] # sensitive
    np.random.seed(0)
    x = np.random.normal(0, 1, size=N) - 0.2*s  # correlated with s

    np.random.seed(1)
    y = 0.3*x + 0.5*s + np.random.normal(0,1,size=N)  # outcome y is correlated with x and s
    y -= np.mean(y)
    y = np.array(logistic(y) > 0.5, dtype=int) # flg

    np.random.seed(2)
    train_idx = np.random.choice(N, size=N//2, replace=False) # random split
    test_idx = np.array(list(set(np.arange(N)) - set(train_idx)))

    return x[train_idx], y[train_idx], s[train_idx], x[test_idx], y[test_idx], s[test_idx]

# 公平性の各種metricsをprint
def print_result_summary(y_true, y_pred, s):
    # Demographic Parityで用いる差
    dp_ave =  y_pred.mean()
    dp_s1 =  y_pred[s==1].mean()
    dp_s0 = y_pred[s==0].mean()
    dp_diff = np.abs(dp_s1 - dp_s0)
    
    # TruePositiveRate Parityで用いる差 (Equalized Opportunity)
    tpr_ave = y_pred[y_true==1].mean()
    tpr_s1 = y_pred[np.all([y_true==1, s==1],axis=0)].mean()
    tpr_s0 = y_pred[np.all([y_true==1, s==0],axis=0)].mean()
    tpr_diff = np.abs(tpr_s1 - tpr_s0)
    
    # FalsePositiveRate Parityで用いる差
    fpr_ave = y_pred[y_true==0].mean()
    fpr_s1 = y_pred[np.all([y_true==0, s==1],axis=0)].mean()
    fpr_s0 = y_pred[np.all([y_true==0, s==0],axis=0)].mean()
    fpr_diff = np.abs(fpr_s1 - fpr_s0)
    
    # print result
    dp_text = f"Demographic Parity:\t\t[mean] {dp_ave:.3f},\t[s=1] {dp_s1:.3f},\t[s=0] {dp_s0:.3f},\t[abs_diff] {dp_diff:.3f}"
    tpr_text = f"TruePositiveRate Parity:\t[mean] {tpr_ave:.3f},\t[s=1] {tpr_s1:.3f},\t[s=0] {tpr_s0:.3f},\t[abs_diff] {tpr_diff:.3f}"
    fpr_text = f"FalsePositiveRate Parity:\t[mean] {fpr_ave:.3f},\t[s=1] {fpr_s1:.3f},\t[s=0] {fpr_s0:.3f},\t[abs_diff] {fpr_diff:.3f}"
    print(dp_text)
    print(tpr_text)
    print(fpr_text)

# Metricsの確認

In [4]:
# Metrics
from fairlearn.metrics import (
    demographic_parity_difference,
    true_positive_rate_difference, 
    false_positive_rate_difference,
    equalized_odds_difference)

In [5]:
# テストデータ
y_true = np.array([0,0,1,1,1,0,0])
y_pred = np.array([0,0,0,1,1,1,1])
s = np.array([1,0,1,0,1,0,0])

In [6]:
# 自作
print_result_summary(y_true, y_pred, s)

Demographic Parity:		[mean] 0.571,	[s=1] 0.333,	[s=0] 0.750,	[abs_diff] 0.417
TruePositiveRate Parity:	[mean] 0.667,	[s=1] 0.500,	[s=0] 1.000,	[abs_diff] 0.500
FalsePositiveRate Parity:	[mean] 0.500,	[s=1] 0.000,	[s=0] 0.667,	[abs_diff] 0.667


In [7]:
# fairlearnでのmetrics実装, 自作funcとの[abs_diff]との一致が確認できる
print("Demographic Parity Diff :\t\t %.3f"% demographic_parity_difference(y_true=y_true, y_pred=y_pred, sensitive_features=s))
print("TruePositiveRate Parity Diff :\t %.3f"% true_positive_rate_difference(y_true=y_true, y_pred=y_pred, sensitive_features=s))
print("FalsePositiveRate Parity Diff :\t %.3f"% false_positive_rate_difference(y_true=y_true, y_pred=y_pred, sensitive_features=s))

# TruePositiveRate DiffとFalsePositiveRate Diffのうち大きい方の値が入る
print("EqualizedOdds Parity Diff :\t\t %.3f"% equalized_odds_difference(y_true=y_true, y_pred=y_pred, sensitive_features=s))

Demographic Parity Diff :		 0.417
TruePositiveRate Parity Diff :	 0.500
FalsePositiveRate Parity Diff :	 0.667
EqualizedOdds Parity Diff :		 0.667


# データ生成

In [8]:
# sample data
x_train, y_train, s_train, x_test, y_test, s_test = make_sample_data(N=10000, p_s=0.1)
X_train = np.c_[x_train, s_train]
X_test = np.c_[x_test, s_test]

# ベースラインモデル, そのままモデルfit

### センシティブ変数も入力

In [9]:
# model fit
baseline_model = LGBMClassifier(random_state=1)
baseline_model.fit(X_train, y_train)

# predict
y_train_pred_baseline = baseline_model.predict(X_train)
y_test_pred_baseline = baseline_model.predict(X_test)

In [10]:
print("Train")
print_result_summary(y_true=y_train, y_pred=y_train_pred_baseline, s=s_train)
print("\nTest")
print_result_summary(y_true=y_test, y_pred=y_test_pred_baseline, s=s_test)

Train
Demographic Parity:		[mean] 0.548,	[s=1] 0.575,	[s=0] 0.304,	[abs_diff] 0.271
TruePositiveRate Parity:	[mean] 0.681,	[s=1] 0.689,	[s=0] 0.580,	[abs_diff] 0.109
FalsePositiveRate Parity:	[mean] 0.411,	[s=1] 0.451,	[s=0] 0.130,	[abs_diff] 0.321

Test
Demographic Parity:		[mean] 0.529,	[s=1] 0.558,	[s=0] 0.270,	[abs_diff] 0.288
TruePositiveRate Parity:	[mean] 0.607,	[s=1] 0.624,	[s=0] 0.372,	[abs_diff] 0.252
FalsePositiveRate Parity:	[mean] 0.451,	[s=1] 0.486,	[s=0] 0.216,	[abs_diff] 0.270


### センシティブ変数をDrop

In [11]:
# model fit
baseline_model_wos = LGBMClassifier(random_state=1)
baseline_model_wos.fit(X_train[:,0].reshape(-1,1), y_train)

# predict
y_train_pred_baseline_wos = baseline_model_wos.predict(X_train[:,0].reshape(-1,1))
y_test_pred_baseline_wos = baseline_model_wos.predict(X_test[:,0].reshape(-1,1))

/usr/local/lib/python3.9/site-packages/lightgbm/basic.py:306: UserWarning: Usage of np.ndarray subset (sliced data) is not recommended due to it will double the peak memory cost in LightGBM.
  warnings.warn("Usage of np.ndarray subset (sliced data) is not recommended "
/usr/local/lib/python3.9/site-packages/lightgbm/basic.py:306: UserWarning: Usage of np.ndarray subset (sliced data) is not recommended due to it will double the peak memory cost in LightGBM.
  warnings.warn("Usage of np.ndarray subset (sliced data) is not recommended "
/usr/local/lib/python3.9/site-packages/lightgbm/basic.py:306: UserWarning: Usage of np.ndarray subset (sliced data) is not recommended due to it will double the peak memory cost in LightGBM.
  warnings.warn("Usage of np.ndarray subset (sliced data) is not recommended "


In [12]:
print("Train")
print_result_summary(y_true=y_train, y_pred=y_train_pred_baseline_wos, s=s_train)
print("\nTest")
print_result_summary(y_true=y_test, y_pred=y_test_pred_baseline_wos, s=s_test)

Train
Demographic Parity:		[mean] 0.563,	[s=1] 0.558,	[s=0] 0.608,	[abs_diff] 0.050
TruePositiveRate Parity:	[mean] 0.682,	[s=1] 0.671,	[s=0] 0.824,	[abs_diff] 0.153
FalsePositiveRate Parity:	[mean] 0.440,	[s=1] 0.435,	[s=0] 0.472,	[abs_diff] 0.037

Test
Demographic Parity:		[mean] 0.543,	[s=1] 0.539,	[s=0] 0.578,	[abs_diff] 0.039
TruePositiveRate Parity:	[mean] 0.608,	[s=1] 0.604,	[s=0] 0.669,	[abs_diff] 0.065
FalsePositiveRate Parity:	[mean] 0.476,	[s=1] 0.468,	[s=0] 0.530,	[abs_diff] 0.062


# 公平性を加味したモデルfit

## [Pre-process] CorrelationRemover

In [13]:
# 相関を除去
corr_remover = CorrelationRemover(sensitive_feature_ids=[1]) # X_trainのうち2列目がsに該当
X_train_rmcorr = corr_remover.fit_transform(X_train)
X_test_rmcorr = corr_remover.transform(X_test)

In [14]:
# model fit
clf = LGBMClassifier()
clf.fit(X_train_rmcorr, y_train)

# predict
y_train_pred_rmcorr = clf.predict(X_train_rmcorr)
y_test_pred_rmcorr = clf.predict(X_test_rmcorr)

In [15]:
print("Train")
print_result_summary(y_true=y_train, y_pred=y_train_pred_rmcorr, s=s_train)
print("\nTest")
print_result_summary(y_true=y_test, y_pred=y_test_pred_rmcorr, s=s_test)

Train
Demographic Parity:		[mean] 0.539,	[s=1] 0.539,	[s=0] 0.540,	[abs_diff] 0.001
TruePositiveRate Parity:	[mean] 0.661,	[s=1] 0.655,	[s=0] 0.736,	[abs_diff] 0.080
FalsePositiveRate Parity:	[mean] 0.413,	[s=1] 0.412,	[s=0] 0.417,	[abs_diff] 0.005

Test
Demographic Parity:		[mean] 0.526,	[s=1] 0.524,	[s=0] 0.544,	[abs_diff] 0.020
TruePositiveRate Parity:	[mean] 0.594,	[s=1] 0.588,	[s=0] 0.674,	[abs_diff] 0.087
FalsePositiveRate Parity:	[mean] 0.458,	[s=1] 0.456,	[s=0] 0.476,	[abs_diff] 0.020


## [In-process] Reduction Method

### Demographic Parity

In [16]:
# model fit
sweep = GridSearch(
                   estimator=LGBMClassifier(random_state=1), # estimator needs `sample_weight` at fit 
                   constraints=DemographicParity(),  # EqualizedOdds, TruePositiveRateParityなども使用可
                   grid_size=50, 
                   grid_limit=1
)
sweep.fit(X_train, y_train, sensitive_features=s_train)

# predict
y_train_pred_indp = sweep.predict(X_train) # 最もDemographicParityが小さいものが選ばれる
y_test_pred_indp = sweep.predict(X_test)

In [17]:
print("Train")
print_result_summary(y_true=y_train, y_pred=y_train_pred_indp, s=s_train)
print("\nTest")
print_result_summary(y_true=y_test, y_pred=y_test_pred_indp, s=s_test)

Train
Demographic Parity:		[mean] 0.544,	[s=1] 0.544,	[s=0] 0.536,	[abs_diff] 0.008
TruePositiveRate Parity:	[mean] 0.669,	[s=1] 0.660,	[s=0] 0.788,	[abs_diff] 0.128
FalsePositiveRate Parity:	[mean] 0.414,	[s=1] 0.420,	[s=0] 0.378,	[abs_diff] 0.042

Test
Demographic Parity:		[mean] 0.524,	[s=1] 0.524,	[s=0] 0.520,	[abs_diff] 0.004
TruePositiveRate Parity:	[mean] 0.591,	[s=1] 0.589,	[s=0] 0.616,	[abs_diff] 0.027
FalsePositiveRate Parity:	[mean] 0.456,	[s=1] 0.454,	[s=0] 0.470,	[abs_diff] 0.016


In [18]:
# 各モデルの予測結果metricsを取得
sweep_preds = [predictor.predict(X_train) for predictor in sweep.predictors_] # 各predictorの予測値を取得
dp_diff_list = [
    demographic_parity_difference(y_train, preds, sensitive_features=s_train)
    for preds in sweep_preds
]

In [19]:
np.sort(dp_diff_list)[:5] #top-5のdemographic_parity_diffを確認, sweep.predictによる結果と同一であることを確認できる

array([0.00844444, 0.01511111, 0.038     , 0.05511111, 0.05911111])

### Equalized Odds

In [20]:
# model fit
sweep = GridSearch(
                   estimator=LGBMClassifier(random_state=1), 
                   constraints=EqualizedOdds(),  # EqualizedOdds, TruePositiveRateParityなども使用可
                   grid_size=50, 
                   grid_limit=1
)
sweep.fit(X_train, y_train, sensitive_features=s_train)

# predict
y_train_pred_ineo = sweep.predict(X_train) # 最もDemographicParityが小さいものが選ばれる
y_test_pred_ineo = sweep.predict(X_test)

In [21]:
print("Train")
print_result_summary(y_true=y_train, y_pred=y_train_pred_ineo, s=s_train)
print("\nTest")
print_result_summary(y_true=y_test, y_pred=y_test_pred_ineo, s=s_test)

Train
Demographic Parity:		[mean] 0.555,	[s=1] 0.560,	[s=0] 0.504,	[abs_diff] 0.056
TruePositiveRate Parity:	[mean] 0.682,	[s=1] 0.675,	[s=0] 0.767,	[abs_diff] 0.091
FalsePositiveRate Parity:	[mean] 0.423,	[s=1] 0.435,	[s=0] 0.339,	[abs_diff] 0.097

Test
Demographic Parity:		[mean] 0.538,	[s=1] 0.545,	[s=0] 0.478,	[abs_diff] 0.067
TruePositiveRate Parity:	[mean] 0.606,	[s=1] 0.609,	[s=0] 0.576,	[abs_diff] 0.033
FalsePositiveRate Parity:	[mean] 0.469,	[s=1] 0.475,	[s=0] 0.427,	[abs_diff] 0.048


## [Post-process] 

### Demographic Parity

In [22]:
# set 
optimizer = ThresholdOptimizer(
    estimator=baseline_model, 
    constraints="demographic_parity", # 他に’{false,true}_{positive,negative}_rate_parity’’equalized_odds’ が使用可能
    prefit=True # 学習済みモデルを渡している場合 True, prefit=Falseでestimator=LGBMClassifier(random_state=1)でも同じ結果
)

# fit optimizer
optimizer.fit(X=X_train,y=y_train, sensitive_features=s_train)
y_test_pred_postdp = optimizer.predict(X_test, sensitive_features=s_test, random_state=20) #乱数固定
y_train_pred_postdp = optimizer.predict(X_train, sensitive_features=s_train, random_state=100) # 乱数固定

In [23]:
print("Train")
print_result_summary(y_true=y_train, y_pred=y_train_pred_postdp, s=s_train)
print("\nTest")
print_result_summary(y_true=y_test, y_pred=y_test_pred_postdp, s=s_test)

Train
Demographic Parity:		[mean] 0.578,	[s=1] 0.575,	[s=0] 0.600,	[abs_diff] 0.025
TruePositiveRate Parity:	[mean] 0.693,	[s=1] 0.689,	[s=0] 0.741,	[abs_diff] 0.051
FalsePositiveRate Parity:	[mean] 0.459,	[s=1] 0.451,	[s=0] 0.511,	[abs_diff] 0.060

Test
Demographic Parity:		[mean] 0.557,	[s=1] 0.558,	[s=0] 0.554,	[abs_diff] 0.004
TruePositiveRate Parity:	[mean] 0.625,	[s=1] 0.624,	[s=0] 0.640,	[abs_diff] 0.015
FalsePositiveRate Parity:	[mean] 0.489,	[s=1] 0.486,	[s=0] 0.509,	[abs_diff] 0.023


### Equalized Odds

In [24]:
optimizer = ThresholdOptimizer(
    estimator=clone(baseline_model), 
    constraints="equalized_odds", 
    prefit=False 
)
optimizer.fit(X=X_train,y=y_train.reshape(-1,1), sensitive_features=s_train)
y_test_pred_posteo = optimizer.predict(X_test, sensitive_features=s_test, random_state=20)
y_train_pred_posteo = optimizer.predict(X_train, sensitive_features=s_train, random_state=20)

/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [25]:
print("Train")
print_result_summary(y_true=y_train, y_pred=y_train_pred_posteo, s=s_train)
print("\nTest")
print_result_summary(y_true=y_test, y_pred=y_test_pred_posteo, s=s_test)

Train
Demographic Parity:		[mean] 0.570,	[s=1] 0.575,	[s=0] 0.530,	[abs_diff] 0.045
TruePositiveRate Parity:	[mean] 0.689,	[s=1] 0.689,	[s=0] 0.684,	[abs_diff] 0.005
FalsePositiveRate Parity:	[mean] 0.449,	[s=1] 0.451,	[s=0] 0.433,	[abs_diff] 0.018

Test
Demographic Parity:		[mean] 0.555,	[s=1] 0.558,	[s=0] 0.528,	[abs_diff] 0.030
TruePositiveRate Parity:	[mean] 0.623,	[s=1] 0.624,	[s=0] 0.610,	[abs_diff] 0.014
FalsePositiveRate Parity:	[mean] 0.486,	[s=1] 0.486,	[s=0] 0.485,	[abs_diff] 0.001
